This part includes correlation_function which aims to find the correlation of the Display correlation relationships as heatmap. Display correlation relationships as a plot over time-period.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#import sys
#sys.path.append('/content/drive/MyDrive/Test')

In [2]:
#!/usr/bin/env python
# coding: utf-8

"""
time_series_analyzer.py
A module for analyzing time series data correlations and outliers,
with options to export data in CSV and JSON formats to specified directories.
"""

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import os

def calculate_correlations(df):
    """
    Calculate correlation matrix for numeric columns in the DataFrame.

    Args:
        df (pd.DataFrame): Input time series data with numeric columns

    Returns:
        pd.DataFrame: Correlation matrix
    """
    return df.corr()

def detect_outliers(df, method='zscore', threshold=3):
    """
    Detect outliers in time series data using specified method.

    Args:
        df (pd.DataFrame): Input time series data
        method (str): 'zscore' or 'iqr'
        threshold (float): Detection sensitivity

    Returns:
        dict: Outlier indices per column {column: [indices]}
    """
    outliers = {}

    for col in df.columns:
        if method == 'zscore':
            z_scores = np.abs(stats.zscore(df[col]))
            outliers[col] = np.where(z_scores > threshold)[0]
        elif method == 'iqr':
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            outliers[col] = df[(df[col] < (q1 - 1.5 * iqr)) |
                               (df[col] > (q3 + 1.5 * iqr))].index.tolist()
    return outliers

def plot_correlation_heatmap(corr_matrix, figsize=(10, 8)):
    """
    Plot correlation matrix as heatmap.

    Args:
        corr_matrix (pd.DataFrame): Correlation matrix
        figsize (tuple): Figure dimensions
    """
    plt.figure(figsize=figsize)
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm',
                fmt='.2f', linewidths=0.5)
    plt.title("Correlation Heatmap")
    plt.show()

def plot_time_series(df, outliers=None, figsize=(12, 6)):
    """
    Plot time series data with optional outlier highlighting.

    Args:
        df (pd.DataFrame): Time series data
        outliers (dict): Outlier indices from detect_outliers()
        figsize (tuple): Figure dimensions
    """
    plt.figure(figsize=figsize)

    for col in df.columns:
        # Plot main series
        plt.plot(df.index, df[col], label=col, alpha=0.7)

        # Highlight outliers if provided
        if outliers and col in outliers:
            outlier_idx = outliers[col]
            plt.scatter(df.index[outlier_idx], df[col].iloc[outlier_idx],
                        color='red', marker='o', s=50,
                        label=f'{col} Outliers', zorder=3)

    plt.title("Time Series with Correlations")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

def export_data(df, name, output_dir=None, export_formats=None):
    """
    Export processed data in specified formats to the given directory.

    Args:
        df (pd.DataFrame): DataFrame to export
        name (str): Base name for the output files
        output_dir (str): Directory path for output files (default: current directory)
        export_formats (list): List of formats to export ('csv', 'json')
    """
    if export_formats is None:
        export_formats = ['csv', 'json']

    # If output_dir is specified, ensure it exists
    if output_dir and not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
            print(f"Created directory: {output_dir}")
        except Exception as e:
            print(f"Error creating directory {output_dir}: {e}")
            output_dir = None

    # Prepare file paths
    if output_dir:
        csv_path = os.path.join(output_dir, f'{name}_ready.csv')
        json_path = os.path.join(output_dir, f'{name}_ready.json')
    else:
        csv_path = f'{name}_ready.csv'
        json_path = f'{name}_ready.json'

    if 'csv' in export_formats:
        df.to_csv(csv_path, index=False)
        print(f"Data saved to {csv_path}")

    if 'json' in export_formats:
        df.to_json(json_path, orient='records', date_format='iso')
        print(f"Data saved to {json_path}")

def analyze_time_series(df, outlier_method='zscore', outlier_threshold=3,
                        generate_plots=True, export_formats=None,
                        export_name=None, output_dir=None):
    """
    Main analysis function combining correlation, outlier analysis, and data export.

    Args:
        df (pd.DataFrame): Input time series data
        outlier_method (str): Outlier detection method
        outlier_threshold (float): Outlier detection sensitivity
        generate_plots (bool): Whether to generate and display plots
        export_formats (list): List of formats to export data ('csv', 'json')
        export_name (str): Base name for exported files, defaults to None (no export)
        output_dir (str): Directory path for output files
    """
    # Correlation analysis
    corr_matrix = calculate_correlations(df)

    if generate_plots:
        plot_correlation_heatmap(corr_matrix)

    # Outlier detection
    outliers = detect_outliers(df, method=outlier_method,
                              threshold=outlier_threshold)

    # Time series visualization with outliers
    if generate_plots:
        plot_time_series(df, outliers=outliers)

    # Export data if requested
    if export_name and export_formats:
        export_data(df, export_name, output_dir, export_formats)

    return corr_matrix, outliers

In [9]:
# Example usage:
if __name__ == "__main__":
    try:
        # Load data from Google Drive path
        df = pd.read_csv('/content/drive/MyDrive/Test/complex.csv').set_index('time')# Change to your own file address

        # Return and print the correlation matrix
        corr_matrix, outliers = analyze_time_series(
            df,
            generate_plots=False,  # If in Jupyter, can set True to see plots
            export_formats=['csv', 'json'],
            export_name='complex_data',
            output_dir='/content/drive/MyDrive/Test/'
        )

        # Print correlation matrix
        print("Correlation Matrix:")
        print(corr_matrix)

        # Save correlation matrix
        corr_df = pd.DataFrame(corr_matrix)
        corr_csv_path = os.path.join('/content/drive/MyDrive/Test/', 'correlation_matrix.csv')
        corr_json_path = os.path.join('/content/drive/MyDrive/Test/', 'correlation_matrix.json')

        # Save CSV
        corr_df.to_csv(corr_csv_path)
        print(f"Correlation matrix saved to {corr_csv_path}")

        # Save Jason
        corr_df.to_json(corr_json_path, orient='split')
        print(f"Correlation matrix saved to {corr_json_path}")

    except Exception as e:
        print(f"Error loading or processing data: {e}")

Data saved to /content/drive/MyDrive/Test/complex_data_ready.csv
Data saved to /content/drive/MyDrive/Test/complex_data_ready.json
Correlation Matrix:
          s1        s2        s3
s1  1.000000  0.152738  0.960249
s2  0.152738  1.000000  0.059872
s3  0.960249  0.059872  1.000000
Correlation matrix saved to /content/drive/MyDrive/Test/correlation_matrix.csv
Correlation matrix saved to /content/drive/MyDrive/Test/correlation_matrix.json
